# Instance creator utility (realtime)

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import sys
import copy
import random

import xml.dom.minidom
import xml.etree.ElementTree as ET

sys.path.append('..')
from models.Fleet import Fleet, from_xml

In [2]:
def TSubElement(parent, tag, attrib={}, text=None, **extra):
    element = ET.SubElement(parent, tag, attrib, **extra)
    if text:
        element.text = text
    return element

## ET handler methods
### Root tree

In [3]:
def new_tree(network_params, fleet_params):
    _instance = ET.Element('instance')
    _network = network_tree(_instance, **network_params)
    _fleet = fleet_tree(_instance, fleet)
    _info = info_tree(_instance)
    
    return ET.ElementTree(_instance)

### Info

In [4]:
def info_tree(root, name='', description=''):
    info = TSubElement(root, 'info')
    _name = TSubElement(info, 'name', text=name)
    _description = TSubElement(info, 'description', text=description)
    return info

### Network

In [5]:
def nodes_tree(network, num_depot, num_customer, num_charging_station, 
               cx_low, cx_high, cy_low, cy_high, 
               request_min, request_max, spent_time_min, spent_time_max,
               tw_low_low, tw_low_high, tw_min_width, tw_max_width, 
               num_cs_tech, max_cs_capacity):
    
    # node ET
    nodes = TSubElement(network, 'nodes')
    
    # depot nodes
    for i in range(num_depot):
        pos_x = 0.0
        pos_y = 0.0
        attr = {'id': str(i), 'type': str(0), 'pos_x': str(pos_x), 'pos_y': str(pos_y)}
        node = TSubElement(nodes, 'node', attrib=attr)

    # customer nodes
    for i in range(num_depot, num_depot+num_customer):
        pos_x = np.random.uniform(cx_low, cx_high)
        pos_y = np.random.uniform(cy_low, cy_high)
        
        attr = {'id': str(i), 'type': str(1), 'pos_x': str(pos_x), 'pos_y': str(pos_y)}
        
        node = TSubElement(nodes, 'node', attrib=attr)
        
        request = np.random.uniform(request_min, request_max)
        node.set('demand', '{:.2f}'.format(request))
        
        spent_time = np.random.uniform(spent_time_min, spent_time_max)
        node.set('spent_time', '{:.2f}'.format(spent_time))
        
        tw_low = np.random.uniform(tw_low_low, tw_low_high)
        node.set('time_window_low', '{:.2f}'.format(tw_low))
        
        tw_upp = tw_low + np.random.uniform(tw_min_width, tw_max_width)
        node.set('time_window_upp', '{:.2f}'.format(tw_upp))

    # CS nodes
    q, quadrants = 0, [0, 2, 1, 3]
    r_min = (cx_high-cx_low)/2
    r_max = (cx_high-cx_low)/5
    for i in range(num_depot+num_customer, num_depot+num_customer+num_charging_station):
        r, angle = np.random.uniform(r_min, r_max), np.random.uniform(quadrants[q]*np.pi/2, (quadrants[q]+1)*np.pi/2)
        pos_x, pos_y = r*np.cos(angle), r*np.sin(angle)
        q = q+1 if q < 3 else 0
        
        attr = {'id': str(i), 'type': str(2), 'pos_x': str(pos_x), 'pos_y': str(pos_y)}
        node = TSubElement(nodes, 'node', attrib=attr)
        
        technology = np.random.randint(1, num_cs_tech+1) # choose among 1 and num_cs_tech
        node.set('technology', str(technology))
        
        #capacity = np.random.randint(1, max_cs_capacity+1) # choose among 1 and max_cs_capacity
        capacity = max_cs_capacity
        node.set('capacity', str(capacity))
    
    return nodes
        

def edges_tree(network, tt_factor, ec_factor, dynamic=False):
    edges = TSubElement(network, 'edges')
    nodes = network.find('nodes')
    
    # Iterate edges
    for i in nodes:
        nodeFrom = TSubElement(edges, 'node_from', attrib={'id': i.get('id')})
        x_from, y_from = float(i.get('pos_x')), float(i.get('pos_y'))    
        for j in nodes:
            nodeTo = TSubElement(nodeFrom, 'node_to', attrib={'id': j.get('id')})
            x_to, y_to = float(j.get('pos_x')), float(j.get('pos_y')) 
            
            if i == j:
                travelTime = 0.
                energyConsumption = 0.
                
            else:
                distance = np.sqrt((x_to-x_from)**2 + (y_to-y_from)**2)
                travelTime = tt_factor*distance
                energyConsumption = ec_factor*distance
                
            nodeTo.set('travel_time', '{:.2f}'.format(travelTime))
            nodeTo.set('energy_consumption', '{:.2f}'.format(energyConsumption))
    
    return edges

def charging_technologies_tree(network, tech_list):
    """
    Adds technology types to network file.
    :param network: the network ET element
    :param tech_list: list of dictionaries defining technologies. Each dictionary in tech_list has the structure {time_point0:soc_point0, ..., time_pointN, soc_pointN}
    """
    number_of_tech = len(tech_list)
    technologies = TSubElement(network, 'technologies')
    _tech_list = [TSubElement(technologies, 'technology', attrib={'type': str(i)}) for i in range(1, number_of_tech+1)]
    
    for _tech, tech in zip(_tech_list, tech_list):
        for chargingTime, battLevel in tech.items():
            breakPoint = TSubElement(_tech, 'breakpoint')
            breakPoint.set('charging_time', str(chargingTime))
            breakPoint.set('battery_level', str(battLevel))
            
    return technologies


    

def network_tree(root, num_depot, num_customer, num_charging_station, 
                  cx_low, cx_high, cy_low, cy_high, 
                  request_min, request_max, spent_time_min, spent_time_max,
                  tw_low_low, tw_low_high, tw_min_width, tw_max_width, 
                  num_cs_tech, max_cs_capacity, tt_factor, ec_factor, tech_list):
    
    network = TSubElement(root, 'network')
    nodes = nodes_tree(network, num_depot, num_customer, num_charging_station,
                       cx_low, cx_high, cy_low, cy_high, 
                       request_min, request_max, spent_time_min, spent_time_max,
                       tw_low_low, tw_low_high, tw_min_width, tw_max_width, 
                       num_cs_tech, max_cs_capacity)
    edges = edges_tree(network, tt_factor, ec_factor)
    technologies = charging_technologies_tree(network, tech_list)
    

### Fleet

In [7]:
def fleet_tree(root, fleet):
    _fleet = TSubElement(root, 'fleet')
    for ev_id, attrib in fleet.items():
        attrib['id'] = str(ev_id)
        _ev = TSubElement(_fleet, 'electric_vehicle', attrib=attrib)
    return

def fleet_assign_customers(root, info):
    _fleet = root.find('fleet')
    for id_ev, customers in info.items():
        for _ev in _fleet:
            if id_ev == _ev.get('id'):
                _assgn_customers = TSubElement(_ev, 'assigned_customers')
                for customer in customers:
                    _customer = TSubElement(_assgn_customers, 'node', attrib={'id':str(customer)})
                break
                

def fleet_update(root, update_info):
    _fleet = root.find('fleet')
    for ev_id, info in update_info.items():
        _ev = _fleet[ev_id-1]
        _previous_sequence = TSubElement(_ev, 'previous_sequence')
        sk_list = info['previous_sequence'][0]
        lk_list = info['previous_sequence'][1]
        for sk, lk in zip(sk_list, lk_list):
            attrib = {'sk':str(sk), 'lk':str(lk)}
            _event = TSubElement(_previous_sequence, 'event', attrib=attrib)
        
        _critical_point = TSubElement(_ev, 'critical_point', info['critical_point'])
            
    


### Folder and file path

In [10]:
num_depot = 1
num_customer = 10
num_cs = 2
num_ev = fleet_size = 2
max_capacity = 4

instanceName = f'{num_customer}C_{num_cs}CS_{num_depot}D_{num_ev}EV_{max_capacity}CAP_HIGHWEIGHT'
#instanceName = '10C_2CS_1D_2EV'
#instanceName = 'test'
folderPath = '../data/XML_files/'+instanceName
filePath = folderPath+'/'+instanceName+'.xml'
filePath_realtime = folderPath+'/'+instanceName+'_realtime.xml'
filePath_already_assigned = folderPath+'/'+instanceName+'_already_assigned.xml'

try:
    os.mkdir(folderPath)
except FileExistsError:
    pass

print('Assignation XML path:', filePath)
print('Assigned XML path   :', filePath_already_assigned)
print('Real-time XML path  :', filePath_realtime)


Assignation XML path: ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT.xml
Assigned XML path   : ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT_already_assigned.xml
Real-time XML path  : ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT_realtime.xml


### Create a new instance or open an existing one
Choose if creating a new one by setting `new_instance` to `True`. In contrary, set it to `False` to open an existing one

In [11]:
new_instance = True
print_pretty = True

# Parameter of new instance
tech1 = {0.0: 0.0, 40.0: 75.0, 60.0: 85.0, 150.0: 100.0}
tech2 = {0.0: 0.0, 15.0: 75.0, 40.0: 80.0, 80.0: 100.0}
tech3 = {0.0: 0.0, 10.0: 75.0, 25.0: 80.0, 40.0: 100.0}
tech_list = [tech1]

net_params = {'num_depot': num_depot, 
              'num_customer': num_customer, 
              'num_charging_station': num_cs,
              'cx_low': -10000., 
              'cx_high': 10000, 
              'cy_low': -10000, 
              'cy_high': 10000,
              'request_min': 0.02, 
              'request_max': 0.08, 
              'spent_time_min': 2.0, 
              'spent_time_max': 9.0,
              'tw_low_low': 60*9., 
              'tw_low_high': 60*12.0,
              'tw_min_width': 60*1.8, 
              'tw_max_width': 60*3.5, 
              'num_cs_tech': len(tech_list), 
              'max_cs_capacity': max_capacity, 
              'tt_factor': 60./(35.*1000.), 
              'ec_factor': 0.00028, 
              'tech_list': tech_list}

ev_attribs = {'max_tour_duration': '300.0', 
              'max_payload': '1.9',
              'weight': '1.5',
              'battery_capacity': '220.0', 
              'alpha_down': '38.0', 
              'alpha_up':'82.0'}

fleet = {x: ev_attribs for x in range(num_ev)}

if new_instance:
    dataTree = new_tree(net_params, fleet)

    # Save tree
    print('Saving to:', folderPath)
    dataTree.write(filePath)
    
    # A random customer assignation    
    customer_assignation = {}

    ids_customer = list(range(1, net_params['num_customer']+1))
    customers_per_car = [int(len(ids_customer) / fleet_size)] * fleet_size

    if len(ids_customer) % fleet_size != 0:
        customers_per_car[-1] = int(len(ids_customer) / fleet_size) + 1

    for i, j in enumerate(customers_per_car):
        print('Car', i, 'must visit', j, 'customer/s')
    print('\n')

    for id_car, num_customers in enumerate(customers_per_car):
        ids_customer_to_visit = []
        for j in range(0, num_customers):
            index = random.randint(0, len(ids_customer) - 1)
            ids_customer_to_visit.append(ids_customer.pop(index))
        print('Car', id_car, 'must visit customers_per_vehicle with ID:', ids_customer_to_visit)
        customer_assignation[str(id_car)] = ids_customer_to_visit

    dataTree_assigned = dataTree
    fleet_assign_customers(dataTree_assigned, customer_assignation)
    dataTree_realtime = dataTree_assigned

    print('Saving assignation to:', filePath_already_assigned)
    dataTree_assigned.write(filePath_already_assigned)
    
    print('Saving realtime to:', filePath_realtime)
    dataTree_realtime.write(filePath_realtime)

    if print_pretty:
        xml_pretty = xml.dom.minidom.parse(filePath_realtime).toprettyxml()
        with open(filePath_realtime, 'w') as file:
            file.write(xml_pretty)
        with open(filePath_already_assigned, 'w') as file:
            file.write(xml_pretty)
            
else:
    dataTree = ET.parse(filePath)
    dataTree_assigned = ET.parse(filePath_already_assigned)
    dataTree_realtime = ET.parse(filePath_realtime)

Saving to: ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT
Car 0 must visit 5 customer/s
Car 1 must visit 5 customer/s


Car 0 must visit customers_per_vehicle with ID: [3, 8, 1, 7, 9]
Car 1 must visit customers_per_vehicle with ID: [5, 10, 6, 4, 2]
Saving assignation to: ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT_already_assigned.xml
Saving realtime to: ../data/XML_files/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT/10C_2CS_1D_2EV_4CAP_HIGHWEIGHT_realtime.xml


## Modify CS capacity

In [8]:
#from_file = filePath_already_assigned
from_file = '../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2_already_assigned.xml'
max_capacity = 4

#instanceName = f'{num_customer}C_{num_cs}CS_{num_depot}D_{num_ev}EV_{max_capacity}CAP'
instanceName = '30C_2CS_1D_6EV_4CAP_V2'
#instanceName = 'test'
folderPath = '../data/GA_implementation_xml/'+instanceName
filePath = folderPath+'/'+instanceName+'.xml'
filePath_realtime = folderPath+'/'+instanceName+'_realtime.xml'
filePath_already_assigned = folderPath+'/'+instanceName+'_already_assigned.xml'

try:
    os.mkdir(folderPath)
except FileExistsError:
    pass

print('Copying data from:', from_file)

print('Assignation XML path:', filePath)
print('Assigned XML path   :', filePath_already_assigned)
print('Real-time XML path  :', filePath_realtime)


Copying data from: ../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2_already_assigned.xml
Assignation XML path: ../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2.xml
Assigned XML path   : ../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2_already_assigned.xml
Real-time XML path  : ../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2_realtime.xml


In [10]:
dataTree = ET.parse(from_file)

_nodes = dataTree.find('network').find('nodes')
for _node in _nodes:
    if _node.get('capacity'):
        _node.set('capacity', str(max_capacity))

print('Saving assigned to:', filePath_already_assigned)
dataTree.write(filePath_already_assigned)

print_pretty = True
if print_pretty:
    xml_pretty = xml.dom.minidom.parse(filePath_already_assigned).toprettyxml()
    with open(filePath_already_assigned, 'w') as file:
        file.write(xml_pretty)

Saving assigned to: ../data/GA_implementation_xml/30C_2CS_1D_6EV_4CAP_V2/30C_2CS_1D_6EV_4CAP_V2_already_assigned.xml


# Using Fleet and Network instances

In [3]:
import sys
sys.path.append('..')

from models.Network import Network
from models.Fleet import Fleet, from_xml

In [ ]:
print_pretty = True

# Parameter of new instance
tech1 = {0.0: 0.0, 40.0: 75.0, 60.0: 85.0, 150.0: 100.0}
tech2 = {0.0: 0.0, 15.0: 75.0, 40.0: 80.0, 80.0: 100.0}
tech3 = {0.0: 0.0, 10.0: 75.0, 25.0: 80.0, 40.0: 100.0}
tech_list = [tech1]

net_params = {'num_depot': num_depot, 
              'num_customer': num_customer, 
              'num_charging_station': num_cs,
              'cx_low': -10000., 
              'cx_high': 10000, 
              'cy_low': -10000, 
              'cy_high': 10000,
              'request_min': 0.001, 
              'request_max': 0.015, 
              'spent_time_min': 2.0, 
              'spent_time_max': 9.0,
              'tw_low_low': 60*9., 
              'tw_low_high': 60*12.0,
              'tw_min_width': 60*1.8, 
              'tw_max_width': 60*3.5, 
              'num_cs_tech': len(tech_list), 
              'max_cs_capacity': max_capacity, 
              'tt_factor': 60./(35.*1000.), 
              'ec_factor': 0.00028, 
              'tech_list': tech_list}

### Network

In [ ]:
network = Network()

In [ ]:
network.write_xml()

### Fleet

In [4]:
ev_attribs = {'max_tour_duration': 5*60.0, 
              'max_payload': 1.5,
              'weight': 1.5,
              'battery_capacity': 220.0, 
              'alpha_down': 38.0, 
              'alpha_up': 82.0}



fleet = Fleet()

In [ ]:
fleet.write_xml()